<a href="https://colab.research.google.com/github/himoyuzuki/colab_analytics/blob/main/1%E6%99%82%E9%96%93%E3%81%94%E3%81%A8%E3%81%AE%E5%A4%A9%E6%B0%97%E3%81%A7%E8%A6%8B%E3%82%8B%E6%97%A5%E6%9C%AC%E3%81%AE%E5%A4%A9%E6%B0%97%E3%81%AE%E5%A4%89%E9%81%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1時間ごとの天気で見る日本の天気の変遷

- 以前、「[日本の天気の変遷](https://colab.research.google.com/drive/1IvZ_fa1NWsDM0uqk59TLPx2mhBGTriKP?usp=sharing)」と題して1950年から2025年までの、東京の月平均気温の変化などを見てみた。
- 結果として、平均気温は60年ほどの間で約1.7℃上昇していることが判明したが、実態はより深刻なのではないかという疑念が残った。
- 特に、ここ10数年での夏場の気温が異常に高い。
- そこで今回は、1時間ごとの気温データを使い、より詳細に分析を試みることにした。

**使用データ**

- ソース: [気象庁|過去の気象データ・ダウンロード](https://www.data.jma.go.jp/risk/obsdl/)
- 地点: 東京
- 項目: 1時間ごとの気温

※Colaboratoryの「ファイル」に、data.csvとして上記データをダウンロード後、加工して日付情報と気温(℃)のみにしたデータを使用




In [ ]:
import os
import glob
import re
import csv
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
df = pd.read_csv('./data.csv', index_col=0)
df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')
df.head()

,気温(℃)
年月日時,
2005-01-01 01:00:00,3.0
2005-01-01 02:00:00,3.0
2005-01-01 03:00:00,2.4
2005-01-01 04:00:00,3.0
2005-01-01 05:00:00,2.9


## 13時（最高気温と思われる時間帯）の気温

In [ ]:
df_13 = df[df.index.hour == 13]
df_13.head()

,気温(℃)
年月日時,
2005-01-01 13:00:00,8.2
2005-01-02 13:00:00,8.7
2005-01-03 13:00:00,12.8
2005-01-04 13:00:00,17.3
2005-01-05 13:00:00,7.9


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_13.index, y=df_13['気温(℃)']))
fig.update_layout(title='東京 13時平均気温の推移')
fig.show()

グラフで見てもいまいち2005年と2025年の違いがわからない。

In [ ]:
# 30℃超えの日数を月ごとにカウント

# 30℃超えの行をTrue/Falseでフィルタ
hot_mask = df_13['気温(℃)'] > 30

# 年・月ごとにグループ化して合計（Trueは1としてカウントされる）
count_over30 = hot_mask.groupby([
    pd.Index(df_13.index.year, name='年'),
    pd.Index(df_13.index.month, name='月')
]).sum()

# 見やすくするためMultiIndexをDataFrame化
count_over30_df = count_over30.reset_index()
count_over30_df.columns = ['年', '月', '30度超え日数']

# DataFrame化 & ピボット
df_plot = count_over30.reset_index(name='30度超え日数')
df_pivot = df_plot.pivot(index='年', columns='月', values='30度超え日数')

# Heatmapで可視化
fig = px.imshow(
    df_pivot,
    labels=dict(x="月", y="年", color="30℃超え日数"),
    x=df_pivot.columns,
    y=df_pivot.index,
    color_continuous_scale='Reds',
    text_auto=True
)

fig.update_layout(
    title="各年・月ごとの30℃超え日数（13時）",
    xaxis_title="月",
    yaxis_title="年"
)

fig.show()

**考察**

- 30℃超えの日数で見ると、2005年から7月、8月の約半分は30℃を超えていたことがわかる。
- しかし2000年以降は、6月でも30℃超えの日がまいと

In [ ]:
# 35℃超えの日数を月ごとにカウント

# 35℃超えの行をTrue/Falseでフィルタ
hot_mask = df_13['気温(℃)'] > 35

# 年・月ごとにグループ化して合計（Trueは1としてカウントされる）
count_over35 = hot_mask.groupby([
    pd.Index(df_13.index.year, name='年'),
    pd.Index(df_13.index.month, name='月')
]).sum()

# 見やすくするためMultiIndexをDataFrame化
count_over35_df = count_over35.reset_index()
count_over35_df.columns = ['年', '月', '35度超え日数']

# DataFrame化 & ピボット
df_plot = count_over35.reset_index(name='35度超え日数')
df_pivot = df_plot.pivot(index='年', columns='月', values='35度超え日数')

# Heatmapで可視化
fig = px.imshow(
    df_pivot,
    labels=dict(x="月", y="年", color="35℃超え日数"),
    x=df_pivot.columns,
    y=df_pivot.index,
    color_continuous_scale='Reds',
    text_auto=True
)

fig.update_layout(
    title="各年・月ごとの35℃超え日数（13時）",
    xaxis_title="月",
    yaxis_title="年"
)

fig.show()